# Check Data Quality of the Files

checking data quality of the files
e.g. if there are redundant files or empty files, etc.

Steps:
- importing Boto because files are on S3

In [1]:
#!pip3 install s3fs boto3 mock --use-feature=2020-resolver
# there is a bug in the feature resolver, that the installation runs forever, even though the installation succeeded.
# you may have to interrupt the kernel and then s3fs is isntalled properly

In [2]:
import boto3
import pandas as pd

In [3]:
def file_list(bucket_name):
    """A function that lists all files (objects) in a given bucket (argument bucket_name) and puts it in a list"""
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket_name)
    return [i.key for i in my_bucket.objects.all()]

In [4]:
file_list('dsti-ml-nlp-class')

['Data/MR_IBRAHIM_ASHAFA_STATEMENT (1).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (2).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (3).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT (4).csv',
 'Data/MR_IBRAHIM_ASHAFA_STATEMENT.csv',
 'Data/MR_TAPIYA_JOHN_BANK_STATEMENT-part2.csv',
 'Data/Mr_Tochukwu_statement-part2.csv',
 'Data/mr_gani_olalekan_statement_July_9.xlsx']

In [5]:
def create_s3_file_uri(bucket_name, file_name):
    """takes a bucket name and filename as argument and forms an s3 uri from it."""
    s3_file_uri = "s3://" + bucket_name + "/" + file_name
    return s3_file_uri

In [6]:
def file_to_pandas(s3_file_uri):
    """put a given file into a pandas dataframe. Can differentiate between .xlsx and .csv"""
    if (s3_file_uri.endswith(".csv")):
        data = pd.read_csv(s3_file_uri)
    elif (s3_file_uri.endswith(".xlsx")):
        data = pd.read_excel(s3_file_uri)
    else:
        print("This Function can handle only .xlsx or .csv files")
        data = None
    return data

In [7]:
create_s3_file_uri("dsti-ml-nlp-class", "Data/mr_gani_olalekan_statement_July_9.xlsx")

's3://dsti-ml-nlp-class/Data/mr_gani_olalekan_statement_July_9.xlsx'

In [8]:
s3_test_uri = create_s3_file_uri("dsti-ml-nlp-class", "Data/mr_gani_olalekan_statement_July_9.xlsx")

In [9]:
df = file_to_pandas(s3_test_uri)

In [10]:
def clean_dataframe(df):
    """takes a Dataframe as input, selects the column as header, which has TXN DATE in first column
    select only the columns that are not nan"""

    #selecting only rows with txn data and after
    tmp_df = df[list(df[df.iloc[:, 0] == "TXN DATE"].index)[0]:]

    #removing first two lines of the dataframe and taking the third column as headers
    df_with_headers = pd.DataFrame(tmp_df)
    df_with_headers.columns = tmp_df.iloc[0].values
    df_with_headers.drop(df_with_headers.index[0], inplace=True)

    #reducing into a new df that has no nan columns
    df_reduced = df_with_headers[[el for el in df.iloc[0].values if isinstance(el, str)]]
    return df_reduced

In [11]:
clean_dataframe(df)

KeyError: "None of [Index(['CUSTOMER NAME\\nADDRESS\\nACCOUNT NUMBER\\nACCOUNT TYPE CURRENCY',\n       'GANNI-OLA SAFES & DOORS LTD\\n17 NASCON RD OPP.JONAS PETROLSTATION IJOKO-OTA\\n1300003939\\nCURRENT NGN',\n       'START DATE\\nEND DATE\\nOPENING BALANCE\\nCLOSING BALANCE',\n       '16-May-2021\\n13-Jun-2021\\n3,789,521.51\\n10,574,995.17'],\n      dtype='object')] are in the [columns]"

In [12]:
s3_test_uri = create_s3_file_uri("dsti-ml-nlp-class", "Data/MR_IBRAHIM_ASHAFA_STATEMENT.csv")

In [21]:
df = file_to_pandas(s3_test_uri)

In [ ]:
def extract(dataframe, columns: list):
    """
    Extract columns from a data frame and return data frame

    :param dataframe: pandas data frame
    :param columns: list of columns to extract
    :return: pandas data frame
    """

    return dataframe[columns]

In [22]:
df2 = extract(df, ["Remarks_processed", "CLASSE"])
df2

,Remarks_processed,CLASSE
0,TRANSFER BETWEEN CUSTOMERS via GAPScom on Baba...,transfert
1,TRANSFER BETWEEN CUSTOMERS via GAPScom on Eken...,transfert
2,TRANSFER BETWEEN CUSTOMERS Via GTWorld Eneke ...,transfert
3,NIBSS Instant Payment Outward 0000132012070917...,transfert
4,COMMISSION 000013201207091758000279938909 NIP ...,commission
...,...,...
1278,Airtime Purchase GTWORLD_QL- 101CT000000000265...,utility
1279,NIBSS Instant Payment Outward 0000132106091420...,transfert
1280,COMMISSION 000013210609142049000172467995 NIP ...,commission
1281,VALUE ADDED TAX 000013210609142049000172467995...,tax


In [24]:
df2.CLASSE.value_counts()


transfert       438
tax             248
utility         242
commission      235
bank_charges     49
cash             38
loan             21
reversal          8
others            4
Name: CLASSE, dtype: int64

In [25]:
df2.Remarks_processed

0       TRANSFER BETWEEN CUSTOMERS via GAPScom on Baba...
1       TRANSFER BETWEEN CUSTOMERS via GAPScom on Eken...
2       TRANSFER BETWEEN CUSTOMERS Via GTWorld Eneke  ...
3       NIBSS Instant Payment Outward 0000132012070917...
4       COMMISSION 000013201207091758000279938909 NIP ...
                              ...                        
1278    Airtime Purchase GTWORLD_QL- 101CT000000000265...
1279    NIBSS Instant Payment Outward 0000132106091420...
1280    COMMISSION 000013210609142049000172467995 NIP ...
1281    VALUE ADDED TAX 000013210609142049000172467995...
1282                                                  NaN
Name: Remarks_processed, Length: 1283, dtype: object

In [59]:
import numpy as np


def lower_columns(dataframe, columns_name, default_value=np.NAN):
    """
    Take a data frame, columns name and return a series of this columns lowering.

    :param dataframe: pandas data frame
    :param columns_name: name of the columns to lower
    :param default_value: value for replace no string value
    :return: return a pandas series
    """
    data = dataframe.copy()
    return data[columns_name].apply(lambda i: i.lower() if isinstance(i, str) else default_value)

In [60]:
df2.loc[:, "Remarks_processed_lower"] = lower_columns(df2, "Remarks_processed")

/home/souff/PycharmProjects/NLP-with-DSTI/venv/lib/python3.9/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


In [61]:
df2["Remarks_processed_lower_remove_non_word"] = df2.Remarks_processed_lower.str.replace('[^a-zA-Z ]', '')
df2

/tmp/ipykernel_111503/4180216546.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df2["Remarks_processed_lower_remove_non_word"] = df2.Remarks_processed_lower.str.replace('[^a-zA-Z ]', '')
/tmp/ipykernel_111503/4180216546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Remarks_processed_lower_remove_non_word"] = df2.Remarks_processed_lower.str.replace('[^a-zA-Z ]', '')


,Remarks_processed,CLASSE,Remarks_processed_lower,Remarks_processed_lower_remove_non_word,Remarks_processed_final
0,TRANSFER BETWEEN CUSTOMERS via GAPScom on Baba...,transfert,transfer between customers via gapscom on baba...,transfer between customers via gapscom on baba...,transfer between customers via gapscom babawur...
1,TRANSFER BETWEEN CUSTOMERS via GAPScom on Eken...,transfert,transfer between customers via gapscom on eken...,transfer between customers via gapscom on eken...,transfer between customers via gapscom ekene n...
2,TRANSFER BETWEEN CUSTOMERS Via GTWorld Eneke ...,transfert,transfer between customers via gtworld eneke ...,transfer between customers via gtworld eneke ...,transfer between customers via gtworld eneke r...
3,NIBSS Instant Payment Outward 0000132012070917...,transfert,nibss instant payment outward 0000132012070917...,nibss instant payment outward via gtworld bab...,nibss instant payment outward via gtworld baba...
4,COMMISSION 000013201207091758000279938909 NIP ...,commission,commission 000013201207091758000279938909 nip ...,commission nip transfer commission for via g...,commission nip transfer commission for via gtw...
...,...,...,...,...,...
1278,Airtime Purchase GTWORLD_QL- 101CT000000000265...,utility,airtime purchase gtworld_ql- 101ct000000000265...,airtime purchase gtworldql ct,airtime purchase gtworldql
1279,NIBSS Instant Payment Outward 0000132106091420...,transfert,nibss instant payment outward 0000132106091420...,nibss instant payment outward via gtworld to...,nibss instant payment outward via gtworld yaku...
1280,COMMISSION 000013210609142049000172467995 NIP ...,commission,commission 000013210609142049000172467995 nip ...,commission nip transfer commission for via g...,commission nip transfer commission for via gtw...
1281,VALUE ADDED TAX 000013210609142049000172467995...,tax,value added tax 000013210609142049000172467995...,value added tax vat on nip transfer for via ...,value added tax vat nip transfer for via gtwor...


In [80]:
def remove_small_word(dataframe, column, value=2):
    """
    Remove the word with small size

    :param dataframe: pandas data frame
    :param column: name of the columns
    :param value: limit size of word to remove
    :return: pandas series
    """
    data = dataframe[column].dropna()
    return data.apply(
        lambda i: " ".join([j for j in i.split(' ') if len(j) > value]) if isinstance(i, str) else None)

In [84]:
df2.loc[:, "Remarks_processed_final"] = remove_small_word(df2, "Remarks_processed_lower_remove_non_word")
df2 = df2[["CLASSE", "Remarks_processed_final"]].dropna()


/home/souff/PycharmProjects/NLP-with-DSTI/venv/lib/python3.9/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


In [102]:
def text_to_vect(data, columns):
    """
    Convert pandas series to list of list for gensim use.
    :param data: pandas data frame
    :param columns: pandas column
    :return: list
    """
    return data[columns].apply(lambda i: [k for k in i.split(" ")]).tolist()

In [103]:
gensim_sentences = text_to_vect(df2, "Remarks_processed_final")

In [89]:
!pip install gensim

In [104]:
import gensim

model = gensim.models.Word2Vec(sentences=gensim_sentences)


In [108]:
# You can't use a word which aren't in the dictionary
model.wv['tax']

array([-0.10328881,  0.16334583,  0.01051014, -0.10607824,  0.08909471,
       -0.28044245,  0.07188383,  0.41982266, -0.21036315, -0.14532638,
       -0.05862461, -0.28726697, -0.04851991,  0.18935427,  0.15843366,
        0.00273193,  0.15240349, -0.00335649,  0.04250571, -0.44154388,
        0.1466879 ,  0.01564201,  0.10682976, -0.17818305, -0.05215204,
        0.05861862, -0.08478298, -0.07679153, -0.08991367,  0.02726344,
        0.25149748, -0.00786174,  0.14026217, -0.24906307, -0.11253436,
        0.1135497 ,  0.14178824,  0.05233049, -0.09647244, -0.15661846,
        0.14681949, -0.13713515,  0.01279491,  0.05852986,  0.15708083,
       -0.02222572, -0.09559073, -0.11553458,  0.13193096,  0.13987166,
        0.06783198, -0.07567728,  0.01043837, -0.0084436 ,  0.05231636,
        0.03340466,  0.09843885,  0.01304249, -0.08215646,  0.03802414,
        0.01552138, -0.00406528,  0.14166923,  0.10932786, -0.11888715,
        0.26020974,  0.09080897,  0.19691391, -0.25692618,  0.31